In [2]:
import random
import pandas as pd

characters = {
    5: ['Akuma Fuyu', 'Ajax', 'Arie', 'Benencino', 'Benivondo', 'Bromine', 'Cherri Bloom', 'Harumi', 'Lěng Dié', 'Liàng Huā', 'Rainbowrio', 'Unnamed Daylight', 'Unnamed Nightlight', 'Zayla'],
    4: ['Adrinna', 'Caitylin', 'Hua Jingyi', 'Sire Bliss', 'Navia', 'Rosalyne', 'Scarlet Moon', 'Uviet'],
    3: ['Akuma Akito', 'Amaris', 'Asiandra Dash', 'Blali Flammia', 'Blueberry Mane', 'Midnight Melody', 'Nightsky Artist', 'Sire Bliss'],
    2: ['Torie', 'Aria', 'Asiandra Angel', 'Curlwhirl', 'Dagger', 'Dead Goth', 'Gunshot', 'Killerlane', 'Silkpetal', 'Toffee', 'Yokata Nao'],
    1: ['Callie', 'Clova Pom', 'Misha', 'Moni', 'Morgana', 'Morgance', 'Osip', 'Yoshino Amaya'],
}


rarity_probabilities = {
    5: 0.01,
    4: 0.05,
    3: 0.14,
    2: 0.30,
    1: 0.50
}


def gacha_pull(pull_count=1, pity_counter=0, max_5_star_pity=90, max_4_star_pity=45, guaranteed_4_star=False):
    pulls = []
    
    for _ in range(pull_count):
        if pity_counter >= max_5_star_pity:
            # Guaranteed 5-Star after 90 pulls
            rarity = 5
            character = random.choice(characters[rarity])
            pulls.append((rarity, character))
            pity_counter = 0
        elif guaranteed_4_star or pity_counter >= max_4_star_pity:
            # Guaranteed 4-Star after 45 pulls for 1-pull cases
            rarity = 4
            character = random.choice(characters[rarity])
            pulls.append((rarity, character))
            pity_counter = 0
        else:
            # Random pull based on defined probabilities
            rand = random.random()
            cumulative_probability = 0
            for rarity, prob in rarity_probabilities.items():
                cumulative_probability += prob
                if rand < cumulative_probability:
                    character = random.choice(characters[rarity])
                    pulls.append((rarity, character))
                    break
            pity_counter += 1  # Increment pity counter

    return pulls, pity_counter

# Function to simulate the pulls and display them in a DataFrame
def simulate_gacha(pull_count, pity_counter, history_df=None):
    pulls = []
    
    # For a 10-pull, guarantee one 4-star in the batch
    if pull_count == 10:
        # First, simulate 9 normal pulls
        normal_pulls, pity_counter = gacha_pull(pull_count=9, pity_counter=pity_counter, guaranteed_4_star=False)
        pulls.extend(normal_pulls)
        
        # Guarantee one 4-star pull for the 10th pull
        guaranteed_4_star = (4, random.choice(characters[4]))  # Guaranteed 4-Star character
        pulls.append(guaranteed_4_star)
    
    else:
        # For 1-pulls, just simulate them as normal with pity logic
        pulls, pity_counter = gacha_pull(pull_count=pull_count, pity_counter=pity_counter, max_4_star_pity=45)
    
    # Convert the pulls into a list of dictionaries for DataFrame format
    pull_data = [{"Pull #": len(history_df) + i + 1, "Rarity": rarity, "Character": character}
                 for i, (rarity, character) in enumerate(pulls)]
    
    # Create or update the DataFrame with new pull results
    new_df = pd.DataFrame(pull_data)
    
    # If there's existing history, append the new pulls to the DataFrame
    if history_df is not None:
        history_df = pd.concat([history_df, new_df], ignore_index=True)
    else:
        history_df = new_df
    
    # Display the DataFrame in a nice format
    print(history_df)
    print(f"Updated Pity Counter: {pity_counter}")
    return history_df, pity_counter

# Main loop for continuous pulls
def gacha_game():
    # Initialize the game state
    pity_counter = 0
    history_df = pd.DataFrame(columns=["Pull #", "Rarity", "Character"])  # DataFrame to store pull results
    
    while True:
        # Ask the user how many pulls they want to perform
        try:
            pull_count = int(input("How many pulls would you like to perform? (1 or 10, type 'exit' to stop): "))
            if pull_count not in [1, 10]:
                print("Invalid input. Please enter 1 or 10.")
                continue
        except ValueError:
            command = input("Type 'exit' to stop or any other key to continue: ").strip().lower()
            if command == 'exit':
                print("Thanks for playing!")
                break
            else:
                print("Invalid input. Please enter a valid number of pulls.")
                continue
        
        # Perform the pulls and update the DataFrame
        history_df, pity_counter = simulate_gacha(pull_count, pity_counter, history_df)

# Start the game
gacha_game()

Would you like to do a one pull or a ten pull? Type exit to stop.10
  Pull # Rarity        Character
0      1      1    Yoshino Amaya
1      2      1             Moni
2      3      3  Nightsky Artist
3      4      1           Callie
4      5      1            Misha
5      6      4       Sire Bliss
6      7      2             Aria
7      8      1             Moni
8      9      2           Toffee
9     10      4         Rosalyne
Pity: 9


KeyboardInterrupt: Interrupted by user